Step 2: Create training, valdation, and set data sets 

Generate train, validation, and test json files from json bugtitle-patch obtained in Step 1. 

In [ ]:
# Create training, validation, test json files with positive and negative instances of bug patches

import json
from collections import defaultdict
import random
     

# bug patch data in in json format

patch_tr = [] # List of training patches
patch_val = [] # List of validation patches
patch_tst = [] # List of test patches
bug_tr = [] # List of training bugs
bug_val = [] # List of validation bugs
bug_tst = [] # List of test bugs
bugpatch_tr = defaultdict(list) # Dictionary of training link-patch
bugpatch_val = defaultdict(list) # Dictionary of validation link-patch
bugpatch_tst = defaultdict(list) # Dictionary of testing link-patch
tstbug_idx = [] # List of test bug indices

with open("bugpatch.json", "r") as fp:
    data = json.load(fp)

ctr = 0
idx = 0
for k,v in data.items():
    ctr = ctr + 1
    if (ctr >= 1 and ctr <= 6): # Split 60% training, 20% valid, and 20% test
        bug_tr.append(k)
        patch_tr.append(v)
        bugpatch_tr[k].append((v,1)) # Matching link-patch
    if (ctr == 7 or ctr == 8):
        bug_tst.append(k)
        patch_tst.append(v)
        bugpatch_tst[k].append((v,1)) # Matching link-patch
        tstbug_idx.append(idx)
    if (ctr == 9 or ctr == 10):
        bug_val.append(k)
        patch_val.append(v)
        bugpatch_val[k].append((v,1)) # Matching link-patch
    if (ctr == 10):
        ctr = 0;
    idx = idx+1

# Data augmentation of training with negative examples
num_negatives = 1 # Number of negative training examples for every positive example

for i, ele in enumerate(bug_tr): # For each bug
    for j in range(num_negatives):
        x = random.randint(0,len(patch_tr)-1) # Select a random patch
        if (x != i): # Ensure not a matching patch
                bugpatch_tr[ele].append((patch_tr[x], 0)) # Add mismatched link-patch

# Data augmentation of test with negative examples

for i, ele in enumerate(bug_tst): # For each bug
    for j in range(num_negatives):
        x = random.randint(0,len(patch_tst)-1) # Select a random patch
        if (x != i): # Ensure not a matching patch
                bugpatch_tst[ele].append((patch_tst[x], 0)) # Add mismatched link-patch
                
# Data augmentation of validation with negative examples

for i, ele in enumerate(bug_val): # For each bug
    for j in range(num_negatives):
        x = random.randint(0,len(patch_val)-1) # Select a random patch
        if (x != i): # Ensure not a matching patch
                bugpatch_val[ele].append((patch_val[x], 0)) # Add mismatched link-patch
                
                
# Flatten the dictionary to a list of tuples
tr = []
for k, v in bugpatch_tr.items():
    for ele in v:
        tr.append({"bugtitle":k, "patch":ele[0], "label":ele[1]})
        
tst = []
for k, v in bugpatch_tst.items():
    for ele in v:
        tst.append({"bugtitle":k, "patch":ele[0], "label":ele[1]})
        
val = []
for k, v in bugpatch_val.items():
    for ele in v:
        val.append({"bugtitle":k, "patch":ele[0], "label":ele[1]})

        
# Training data : formatted as needed by torchtext
with open('bugpatchlabel_tr.json', 'w+', encoding="utf-8") as fp:
    for ele in tr:
        json.dump(ele, fp)
        fp.write('\n')


# Validation data : formatted as needed by torchtext
with open('bugpatchlabel_val.json', 'w+', encoding="utf-8") as fp:
    for ele in val:
        json.dump(ele, fp)
        fp.write('\n')

# Test data : formatted as needed by torchtext
with open('bugpatchlabel_tst.json', 'w+', encoding="utf-8") as fp:
    for ele in tst:
        json.dump(ele, fp)
        fp.write('\n')

# Test data indices. Needed for evaluating rank
with open('testindex.txt', 'w+', encoding="utf-8") as fp:
    fp.writelines(["%d\n" % i for i in tstbug_idx])  


